In [9]:
%pip install boto3
import boto3
import pandas as pd
from io import StringIO

In [ ]:
AWS_ACCESS_KEY=""
AWS_SECRET_KEY=""
AWS_REGION="ap-south-1"
SCHEMA_NAME="covid_19"
S3_STAGING_DIR=""
S3_BUCKET_NAME=""
S3_OUTPUT_DIRECTORY=""

### Connect to Athena

In [ ]:
athena_client= boto3.client(
    "athena",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION
)

In [ ]:
Dict= {}
def download_and_load_query_results(
    client: boto3.client, query_response: Dict
) -> pd.DataFrame:
    while True:
      try:
        #This function only loads the first 1000 rows
        client.get_query_results(
            QueryExecutionId=query_response["QueryExecutionId"]
        )
        break
    except Exception as err:
      if "not yet finished" in str(err):
        time.sleep(0.001)
      else:
        raise err
    time_file_location: str="athena_query_resulys.csv"
    s3_client= boto3.client(
        "s3",
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        region_name=AWS_REGION,
    )
    s3_client.download.file(
        S3_BUCKET_NAME,
        f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
        temp_file_location,
    )
    return pd.read.csv(temp_file_location)

### Import dataset into pandas dataframe

In [ ]:
response= athena_client.start_query_execution(
    QueryString="SELECT * FROM enigma_jhud",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption":"SSE_S3"},
    },
)

In [ ]:
enigma_jhud= download_and_load_query_results(athena_client,response)

In [ ]:
response= athena_client.start_query_execution(
    QueryString="SELECT * FROM nytimes_data_in_usa_us_county",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption":"SSE_S3"},
    },
)
nytimes_data_in_usa_us_county= download_and_load_query_results(athena_client,response)

In [ ]:
response= athena_client.start_query_execution(
    QueryString="SELECT * FROM nytimes_data_in_usa_us_states",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption":"SSE_S3"},
    },
)
nytimes_data_in_usa_us_states= download_and_load_query_results(athena_client,response)

In [ ]:
response= athena_client.start_query_execution(
    QueryString="SELECT * FROM static_datacountrycode",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption":"SSE_S3"},
    },
)
static_datacountrycode= download_and_load_query_results(athena_client,response)

In [ ]:
# run several query in athena
response= athena_client.start_query_execution(
    QueryString="SELECT * FROM static_datacountrypopulation",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption":"SSE_S3"},
    },
)
static_datacountrypopulation= download_and_load_query_results(athena_client,response)

In [ ]:
response= athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_covid_19_testing_dstatus_daily",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption":"SSE_S3"},
    },
)
rearc_covid_19_testing_dstatus_daily= download_and_load_query_results(athena_client,response)

In [ ]:
response= athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_usa_hospital_beds",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption":"SSE_S3"},
    },
)
rearc_usa_hospital_beds= download_and_load_query_results(athena_client,response)

In [ ]:
response= athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_covid_19_testing_dstatus_total_latest",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption":"SSE_S3"},
    },
)
rearc_usa_hospital_beds= download_and_load_query_results(athena_client,response)

In [ ]:
response= athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_covid_19_testing_data_states_dailystates_daily",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption":"SSE_S3"},
    },
)
rearc_covid_19_testing_data_states_dailystates_daily= download_and_load_query_results(athena_client,response)

In [ ]:
response= athena_client.start_query_execution(
    QueryString="SELECT * FROM static_datastate_abv",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption":"SSE_S3"},
    },
)
static_datastate_abv= download_and_load_query_results(athena_client,response)

In [ ]:
new_header= static_datastate_abv.iloc[0]
static_datastate_abv= static_datastate_abv[1:]
static_datastate_abv.columns=new_header

### Create Star Schema

In [ ]:
factCovid_1= enigma_jhud[['fips','province_state','country_region','confirmed','deaths','recovered','active']]
factCovid_2= rearc_covid_19_testing_data_states_dailystates_daily[['fips','date','positive','negative','hospitalizedcurrently','hospitalized','hospitalizeddischarged']]
factCovid= pd.merge(factCovid_1,factCovid_2, on='fips',how='inner')

In [ ]:
dimRegion_1= enigma_jhud[['fips','provice_state','latitude','longitude']]
dimRegion_2= nytimes_data_in_usa_us_country[['fips','county','state']]
dimRegion= pd.merge(dimRegion_1,dimRegion_2, on='fips', how='inner')

In [ ]:
dimHospital= rearc_usa_hospital_beds[['fips','state_name','latitude','longitude','hq_address','hospital_name','hospital_type','hq_city','hq_state']]

In [ ]:
dimDate= rearc_covid_19_testing_data_states_dailystates_daily[['fips','date',]]

In [ ]:
dimDate['date']= pd.to_datetime(dimDate['date'], format='%Y%m%d')

In [ ]:
dimDate['year']= dimDate['date'].dt.year
dimDate['month']= dimDate['date'].dt.month
dimDate['day_of_week']= dimDate['date'].dt.dayofweek

### Export schema to S3

In [ ]:
bucket= 'zaid-covid-de-project'

In [ ]:
csv_buffer= StringIO()
factCovid.to_csv(csv_buffer)
s3_resource=boto3.resource('s3')
s3_resouce.Object(bucket,'output/factCovid.csv').put(Body=csv_buffer.getvalue())

In [ ]:
csv_buffer= StringIO()
dimDate.to_csv(csv_buffer)
s3_resource=boto3.resource('s3')
s3_resouce.Object(bucket,'output/dimDate.csv').put(Body=csv_buffer.getvalue())

In [ ]:
csv_buffer= StringIO()
dimHospital.to_csv(csv_buffer)
s3_resource=boto3.resource('s3')
s3_resouce.Object(bucket,'output/dimHospital.csv').put(Body=csv_buffer.getvalue())

In [ ]:
csv_buffer= StringIO()
dimRegion.to_csv(csv_buffer)
s3_resource=boto3.resource('s3')
s3_resouce.Object(bucket,'output/dimRegion.csv').put(Body=csv_buffer.getvalue())

In [ ]:
dimDatasql= pd.io.sql.get_schema(dimDate.reset_index(),'dimDate')
print(''.join(dimDatesql))

In [ ]:
factCovidsql= pd.io.sql.get_schema(factCovid.reset_index(),'factCovid')
print(''.join(factCovidsql))

In [ ]:
dimRegionsql= pd.io.sql.get_schema(dimRegion.reset_index(),'dimRegion')
print(''.join(dimRegionsql))

In [ ]:
dimHospitalsql= pd.io.sql.get_schema(dimHospital.reset_index(),'dimHospital')
print(''.join(dimHospitalsql))

### Connect to Redshift

In [ ]:
%pip install redshift-connector
import redshift_connecter

In [ ]:
conn= redshift_connecter.connect(
    host='',
    database='',
    user='',
    password=''
)

In [ ]:
conn.autommit= True

In [ ]:
cursor= redshift_connecter.Cursor= conn.cursor()

In [ ]:
cursor.excecute("""
CREATE TABLE "dimDate" (
  "index" INTEGER,
  "fips" INTEGER,
  "date" TIMESTAMP,
  "year" INTEGER,
  "month" INTEGER,
  "day_of_week" INTEGER
)
""")

cursor.excecute("""
CREATE TABLE "dimHospital" (
  "index" INTEGER,
  "fips" INTEGER,
  "state_name" TEXT,
  "latitude" REAL,
  "longitude" REAL,
  "hd_address" TEXT,
  "hospital_name" TEXT,
  "hospital_type" TEXT,
  "hq_city" TEXT,
  "hq_state" TEXT
)
""")

cursor.excecute("""
CREATE TABLE "factCovid" (
  "index" INTEGER,
  "fips" INTEGER,
  "province_state" TEXT,
  "country_region" TEXT,
  "confirmed" REAL,
  "deaths" REAL,
  "recovered" REAL,
  "active" REAL,
  "date" INTEGER,
  "positive" REAL,
  "negative" REAL,
  "hospitalizedcurrently" REAL,
  "hospitalized" REAL,
  "hospitalizeddischarged" REAL
)
""")

cursor.excecute("""
CREATE TABLE "factCovid" (
  "index" INTEGER,
  "fips" INTEGER,
  "province_state" TEXT,
  "country_region" TEXT,
  "latitude" REAL,
  "longitude" REAL,
  "county" TEXT,
  "state" TEXT
)
""")

In [ ]:
cursor.excecute("""
copy dimData from ''
credentials ''
delimiter ','
region 'ap-south-1'
IGNOREHEADER 1
""")